In [2]:
!pip install pystac
!pip install pystac-client

In [3]:
import pystac
import pystac_client
import random
import folium
from typing import Dict, List

In [4]:
# Credentials and details for Azure APIM
APIM_STAC_URL = "https://os-eo-platform-rg-prod-apim.azure-api.net/stac"
APIM_AUTH_TOKEN = "d208843ea70a43d29416315322dc3228"
# Create AUTH Header object
auth_headers = {"ocp-apim-subscription-key": APIM_AUTH_TOKEN}

In [5]:
# Create Pystac client object
client: pystac_client.client.Client = pystac_client.Client.from_file(
    APIM_STAC_URL, headers=auth_headers
)

In [6]:
# Filter items
bbox: List[float] = [-1.58, 50.52, -1.00, 50.78]
COLLECTIONS = ["landsat-c2-l2", "os_rgbi"]
cql_filter_dict: Dict[str, str] = {
    "op": "or",
    "args": [
        {
            "op": "<=",
            "args": [{"property": "eo:cloud_cover"}, 10.00],
        },
        {
            "op": "=",
            "args": [{"property": "os-rgbi:flying-block"}, "IOW01_23"],
        },
    ],
}
items: pystac.ItemCollection = client.search(
    collections=COLLECTIONS, filter=cql_filter_dict, bbox=bbox
)
for item in items.item_collection():
    item: pystac.Item = item
    print(item.id)

LC09_L2SP_201025_20230907_02_T1
LC08_L2SP_202024_20230906_02_T1
fff4de9e-1dac-4da4-8bd5-a4bdd3d9a59c
ffe20cc3-1767-496c-bea1-013294d6aa65
ffc02d13-13f5-4efb-bab8-7ed5d12d6a8c
febbe0f3-2a4f-4f53-9bb3-4fe98bc96eed
fdb8b2b9-8015-4bf4-8bfa-12b22485b47e
fd90fe9e-bbbb-409f-9ff7-a79c68c681e2
fd898351-4ebd-447b-b4b7-ae3884df1de8
fd218c8b-7dc3-4914-b2a5-903b29050b27
fcf171d7-07b3-4be4-ab15-22087a6c4578
fcb395d4-4df7-405d-a494-edd3b1bb99a7
fc8d8b06-114c-43f9-a622-68dd26d263cb
fc3df7e4-ffbe-4bcd-b6a1-31f2898dc0f2
faaa08f6-1926-4997-be1a-fa12221ffd69
fa74034b-7520-42fd-9ce7-87d3b4720f70
fa652acd-d9ff-4808-a4d3-3668125b8d57
fa48dcfb-48a5-450b-a70a-97211a57512f
f97a8cc9-27a4-4e89-97d5-59505738e5c3
f8e38197-49ae-4aa1-a6ce-882a3b9b53ca
f8007666-f963-44e1-9b1c-3cf468d95590
f7c7e5d5-3808-41f5-851b-bb42f4bd6f4f
f78c712a-b456-4ea7-981f-cfb36e414e87
f6155f3a-4ed9-4d4b-8aee-e0ce1d9428d0
f5fe3853-6154-4c2f-a5d4-1e13a632c901
f3d43c16-5e32-4758-ac9a-9b7d7f172e67
f35841ca-c607-4e8f-a105-b9eaa9949c71
f3108efa-92

In [7]:
# plot all item bboxes on a map
first_item: pystac.Item = items.item_collection()[0]

bbox_for_map: Dict = (
    first_item.bbox[1],
    first_item.bbox[0],
    first_item.bbox[3],
    first_item.bbox[2],
)
print(bbox_for_map)

folium_map: folium.Map = folium.Map(
    tiles="OpenStreetMap", zoom_start=6, location=[bbox_for_map[0], bbox_for_map[1]]
)


def random_color() -> str:
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))


for item in items.item_collection():
    item: pystac.Item = item
    bbox: Dict = (item.bbox[1], item.bbox[0], item.bbox[3], item.bbox[2])
    # map bbox on folium map
    folium.Rectangle(
        bounds=[[bbox[0], bbox[1]], [bbox[2], bbox[3]]],
        fill=False,
        tooltip=item.get_self_href(),
        fill_color=random_color(),
        popup=item.get_self_href(),
    ).add_to(folium_map)

folium_map

(49.172755167814714, -1.8240057454490604, 51.35854483218529, 1.548805005235313)
